The first part of this demo is exactly the same as in *slide_around_demo*. You need to refer to that demo for details.

In [3]:
import os
import sys
import collections
import pickle

# Import third party libraries
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

# Import local libraries
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
a = os.path.join(module_path, "strands_qsr_lib\qsr_lib\src3")

sys.path.append(a)

import plotting
from rl import action_learner, action_learner_search, value_estimator, discrete_action_learner_search as dal
import progress_learner
import config
import project
# Need to add this import to load class
from project import Project
from rl import block_movement_env
import utils

p = project.Project.load(os.path.join('..', utils.MODEL_PATH, 'slidearound_project.proj'))
c = config.Config()
c.progress_threshold=0.92
c.keep_branching = 24
c.branching = 24

tf.reset_default_graph()

global_step = tf.Variable(0, name="global_step", trainable=False)

sess =  tf.Session()

with tf.variable_scope("search") as scope:
    policy_estimator = action_learner_search.PolicyEstimator(c)

sess.run(tf.global_variables_initializer())

with tf.variable_scope("model") as scope:
    print('-------- Load progress model ---------')
    progress_estimator = progress_learner.EventProgressEstimator(is_training=False, name = p.name, config = c)  

# Print out all variables that would be restored
for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'):
    print (variable.name)

saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'))
saver.restore(sess, os.path.join('..', utils.MODEL_PATH, 'progress_SlideAround.mod'))

-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
model/SlideAround/linear/weight:0
model/SlideAround/linear/bias:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0
model/SlideAround/output_linear/weight:0
model/SlideAround/output_linear/bias:0
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideAround.mod


In [ ]:
Using 

We load a 

In [5]:
feedback = "..\\experiments\\human_evaluation_2d\\slidearound.txt"

with open(feedback, 'r') as fh:
    bad_demonstration_indices = []
    good_demonstration_indices = []
    for line in fh:
        values = [int(v) for v in line.split()]
        index = values[0]
        average_score = np.mean(values[1:])
        if average_score <= 2.5:
            bad_demonstration_indices.append(index)
        if average_score >= 6:
            good_demonstration_indices.append(index)

print (len(bad_demonstration_indices))
print (bad_demonstration_indices)
print (len(good_demonstration_indices))
print (good_demonstration_indices)

19
[0, 2, 3, 4, 6, 8, 9, 11, 12, 16, 17, 20, 21, 23, 24, 25, 27, 28, 29]
4
[10, 13, 18, 19]


In [ ]:
prefix = os.path.join( "..", "experiments", "human_evaluation_2d" , "SlideAround")

